# Import of licenced vehicle data in Germany

From ERGÄNZE AMT the data for licenced cars and vehicles in different locations throughout Germany should be imported.
The original data was stored in non uniformly formatted pdf files. For easier input the relevant data was copied into a table editor and after short rearrangement saved as csv. 

Data for year had to be taken from the filename, bundesland and rb were extra rows, which needed to be added as new columns to the remaining data.

In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import glob
%matplotlib inline

In [2]:
paths = glob.glob("./kfz_csv/*.csv")

In [3]:
# read in data file by file
dfs = []
for path in paths:
    #get year from filename
    year = int(path.replace("\\", "/").split("/")[-1].strip(".csv"))
    #print(year)
    land = ""
    rb = ""
    data = []
    ofile = open(path,"r")
    header = ofile.readline()
    
    # parse file
    for line in ofile:
        # 'content' rows start with a digit
        if line[0].isdigit():
            data.append([year,land, rb] + line.replace("\n","").split(";"))
        
        # RB rows start with 'RB'
        elif line.startswith("RB"):
            rb = line.split(" ")[1].replace("\n","").split(";")[0]
        
        # otherwise it is bundesland
        else:
            land = line.replace("\n","").split(";")[0]
    ofile.close()
    
    # combine all gathered rows to a dataframe
    df = pd.DataFrame(data, columns = ["year","land", "rb"] + header.replace("\n","").split(";"))
    # save df in list
    dfs.append(df)

In [4]:
# combine all dfs
df = pd.concat(dfs)

In [5]:
df

,year,land,rb,Stadt,Insgesamt,Benzin,Diesel,Gas (einschl. bivalent),Hybrid,Elektro,...,Euro 5,Euro 6,Sonstige2,Euro 1 2,Euro 2 2,Euro 3 2,Euro 4 2,Euro 5 2,Euro 6 2,Sonstige3
0,2012,SCHLESWIG-HOLSTEIN,,"01001 FLENSBURG,STADT",39433,26375,12601,448,,,...,5294,9,187,233,1667,3239,4692,2541,9,97
1,2012,SCHLESWIG-HOLSTEIN,,"01002 KIEL,STADT",99724,71185,27538,986,,,...,13421,30,836,679,3857,6995,9385,5991,30,427
2,2012,SCHLESWIG-HOLSTEIN,,"01003 LUEBECK,STADT",89654,65879,22799,1133,,,...,10900,30,646,551,3147,5919,7969,4727,30,307
3,2012,SCHLESWIG-HOLSTEIN,,"01004 NEUMUENSTER,STADT",37974,27051,10435,604,,,...,4441,9,165,204,1439,2791,3876,1960,9,71
4,2012,SCHLESWIG-HOLSTEIN,,01051 DITHMARSCHEN,72836,48921,23173,775,,,...,6780,21,518,539,4421,6876,8004,2912,21,253
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
395,2019,THUERINGEN,ZUSAMMEN,16073 SAALFELD-RUDOLSTADT,60.204,41.610,17.795,,,,...,,,,,,,,,,
396,2019,THUERINGEN,ZUSAMMEN,16074 SAALE-HOLZLANDKREIS,49.320,33.522,14.962,,,,...,,,,,,,,,,
397,2019,THUERINGEN,ZUSAMMEN,16075 SAALE-ORLA-KREIS,50.304,33.773,15.944,,,,...,,,,,,,,,,
398,2019,THUERINGEN,ZUSAMMEN,16076 GREIZ,60.390,41.612,17.927,,,,...,,,,,,,,,,


* missing values in later columns
* for analysis missing values not important

In [6]:
df["Insgesamt"].head(5)

0    39433
1    99724
2    89654
3    37974
4    72836
Name: Insgesamt, dtype: object

* check 'bundesland' values

### Data cleaning

* check 'land' column
* conform to other data

In [7]:
df["land"].unique()

array(['SCHLESWIG-HOLSTEIN', 'HAMBURG', 'NIEDERSACHSEN', 'BREMEN',
       'NORDRHEIN-WESTFALEN', 'HESSEN', 'RHEINLAND-PFALZ',
       'BADEN-WUERTTEMBERG', 'BAYERN', 'SAARLAND', 'BERLIN',
       'BRANDENBURG', 'MECKLENBURG-VORPOMMERN', 'SACHSEN',
       'SACHSEN-ANHALT', 'THUERINGEN', 'BERLIN ', 'BRANDENBURG ',
       'BREMEN ', 'HAMBURG ', 'HESSEN ', 'MECKLENBURG-VORPOMMERN ',
       'NIEDERSACHSEN ', 'NORDRHEIN-WESTFALEN ', 'RHEINLAND-PFALZ ',
       'SAARLAND ', 'SACHSEN ', 'SACHSEN-ANHALT ', 'SCHLESWIG-HOLSTEIN ',
       'THUERINGEN '], dtype=object)

* strip leading and following space characters

In [8]:
df["land"] = df["land"].str.strip(" ")

In [9]:
df[df["land"]=="SACHSEN-ANHALT"]

,year,land,rb,Stadt,Insgesamt,Benzin,Diesel,Gas (einschl. bivalent),Hybrid,Elektro,...,Euro 5,Euro 6,Sonstige2,Euro 1 2,Euro 2 2,Euro 3 2,Euro 4 2,Euro 5 2,Euro 6 2,Sonstige3
364,2012,SACHSEN-ANHALT,ZUSAMMEN,"15001 DESSAU-ROSSLAU,STADT",42134,33850,7632,700,,,...,5123,9,41887,88,699,2096,3120,1547,9,49
365,2012,SACHSEN-ANHALT,ZUSAMMEN,"15002 HALLE (SAALE),STADT",89588,71564,16641,1361,,,...,12078,28,88978,162,1290,4162,6953,3912,28,73
366,2012,SACHSEN-ANHALT,ZUSAMMEN,"15003 MAGDEBURG, STADT",103059,80614,21249,1212,,,...,14085,41,101278,220,1744,5209,8311,5507,41,148
367,2012,SACHSEN-ANHALT,ZUSAMMEN,15081 ALTMARKKREIS SALZWEDEL,50285,36980,12719,575,,,...,4425,5,49789,195,1840,3793,4907,1816,5,116
368,2012,SACHSEN-ANHALT,ZUSAMMEN,15082 ANHALT-BITTERFELD,93368,74688,17802,854,,,...,10121,10,92655,212,1761,5000,7362,3315,10,83
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
357,2019,SACHSEN-ANHALT,ZUSAMMEN,15087 MANSFELD-SUEDHARZ,79.807,56.375,22.499,,,,...,,,,,,,,,,
358,2019,SACHSEN-ANHALT,ZUSAMMEN,15088 SAALEKREIS,113.461,80.909,30.686,,,,...,,,,,,,,,,
359,2019,SACHSEN-ANHALT,ZUSAMMEN,15089 SALZLANDKREIS,105.119,76.090,27.397,,,,...,,,,,,,,,,
360,2019,SACHSEN-ANHALT,ZUSAMMEN,15090 STENDAL,63.397,41.547,20.991,,,,...,,,,,,,,,,


* drop non-needed columns

In [10]:
df.drop(axis=1, columns = header.replace("\n","").split(";")[4:], inplace=True)

In [11]:
df[["Insgesamt", "Benzin", "Diesel"]].describe()

,Insgesamt,Benzin,Diesel
count,3205,3205,3205
unique,3186,3183,3166
top,137.811,34308,35760
freq,2,2,2


* numbers contain '.' for readability --> 10000 as 10.000
* remove '.' so it can be read as float

In [12]:
for head in header.replace("\n","").split(";")[1:4]:
    # remove "." and turn numeric
    df[head] = pd.to_numeric(df[head].str.replace(".", ""))
    

In [13]:
df["Insgesamt"]

0      39433
1      99724
2      89654
3      37974
4      72836
       ...  
395    60204
396    49320
397    50304
398    60390
399    51330
Name: Insgesamt, Length: 3205, dtype: int64

* split "Stadt" column into "kennzahl" and "ort" 
* by splitting each row by " " (space)
* add as new columns

In [14]:
def splits(val):
    return val[:5]
    
df["kennzahl"] = df["Stadt"].apply(splits)

In [15]:
def splits2(val):
    return val[5:].strip(" ")
    
df["ort"] = df["Stadt"].apply(splits2)

In [16]:
df["kennzahl"]

0      01001
1      01002
2      01003
3      01004
4      01051
       ...  
395    16073
396    16074
397    16075
398    16076
399    16077
Name: kennzahl, Length: 3205, dtype: object

In [17]:
df["ort"]

0          FLENSBURG,STADT
1               KIEL,STADT
2            LUEBECK,STADT
3        NEUMUENSTER,STADT
4             DITHMARSCHEN
              ...         
395    SAALFELD-RUDOLSTADT
396    SAALE-HOLZLANDKREIS
397       SAALE-ORLA-KREIS
398                  GREIZ
399       ALTENBURGER LAND
Name: ort, Length: 3205, dtype: object

In [18]:
df

,year,land,rb,Stadt,Insgesamt,Benzin,Diesel,kennzahl,ort
0,2012,SCHLESWIG-HOLSTEIN,,"01001 FLENSBURG,STADT",39433,26375,12601,01001,"FLENSBURG,STADT"
1,2012,SCHLESWIG-HOLSTEIN,,"01002 KIEL,STADT",99724,71185,27538,01002,"KIEL,STADT"
2,2012,SCHLESWIG-HOLSTEIN,,"01003 LUEBECK,STADT",89654,65879,22799,01003,"LUEBECK,STADT"
3,2012,SCHLESWIG-HOLSTEIN,,"01004 NEUMUENSTER,STADT",37974,27051,10435,01004,"NEUMUENSTER,STADT"
4,2012,SCHLESWIG-HOLSTEIN,,01051 DITHMARSCHEN,72836,48921,23173,01051,DITHMARSCHEN
...,...,...,...,...,...,...,...,...,...
395,2019,THUERINGEN,ZUSAMMEN,16073 SAALFELD-RUDOLSTADT,60204,41610,17795,16073,SAALFELD-RUDOLSTADT
396,2019,THUERINGEN,ZUSAMMEN,16074 SAALE-HOLZLANDKREIS,49320,33522,14962,16074,SAALE-HOLZLANDKREIS
397,2019,THUERINGEN,ZUSAMMEN,16075 SAALE-ORLA-KREIS,50304,33773,15944,16075,SAALE-ORLA-KREIS
398,2019,THUERINGEN,ZUSAMMEN,16076 GREIZ,60390,41612,17927,16076,GREIZ


Final form achieved

In [19]:
df["ort"]

0          FLENSBURG,STADT
1               KIEL,STADT
2            LUEBECK,STADT
3        NEUMUENSTER,STADT
4             DITHMARSCHEN
              ...         
395    SAALFELD-RUDOLSTADT
396    SAALE-HOLZLANDKREIS
397       SAALE-ORLA-KREIS
398                  GREIZ
399       ALTENBURGER LAND
Name: ort, Length: 3205, dtype: object

Save resulting dataframe as pickle

In [159]:
df.to_pickle("kfz_data.pkl")